In [1]:
from textwrap import indent

from IPython.display import Javascript

from liblet import ANTLR, AnnotatedTreeWalker, Tree, pyast2tree, side_by_side

from operator import add, mul, truediv, sub

from operator import itemgetter

from IPython.display import Latex

from fractions import Fraction

import importlib

import core.arithmetic_manager

from core.util import *

In [7]:
importlib.reload(core.arithmetic_manager)
ArithManager = core.arithmetic_manager.ArithManager

In [11]:
src = '(2 + 2 ) - 3'


arithExpr = ArithManager.arith_expr('antlrGrammar.g')
ast = ArithManager.ptree2ast(ArithManager.ptree(arithExpr, '2 + 2'))
blocks = ArithManager.generate_blocks(ast)
MEMORY = dict(blocks)

main_block = blocks[-1][1]


while blocks:
    block_id, current_block = blocks[0]
    parent_to_calc = ArithManager.annotate_with_priority(current_block)
    #print(100 * '*', '\n', MEMORY)
    tex, _ = ArithManager.latex_format(main_block, {})
    display_latex(tex)
    parent_to_calc.children = [Tree({'type': 'atomExpr', 'value': ArithManager.eval(child), 'priority': 0, '_calc': 'last'}, [])
                               if is_next_to_calc(child) 
                               else child 
                               for child in parent_to_calc.children]
    if is_calculable(current_block):
        parent_to_calc = parent_to_calc.children[0]
        blocks = blocks[1:]
    #display(current_block)
    
    MEMORY[block_id] = parent_to_calc

    
tex, _ = ArithManager.latex_format(main_block, {})
display_latex(tex)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:

main_block = blocks[-1][1]

while blocks:
    block_id, current_block = blocks[0]
    parent_to_calc = atw_prior(current_block)
    #print(100 * '*', '\n', MEMORY)
    tex, _ = atw_latex_formatter(main_block)
    display_latex(tex)
    parent_to_calc.children = [Tree({'type': 'atomExpr', 'value': atw_eval(child), 'priority': 0, '_calc': 'last'}, [])
                               if is_next_to_calc(child) 
                               else child 
                               for child in parent_to_calc.children]
    if is_calculable(current_block):
        parent_to_calc = parent_to_calc.children[0]
        blocks = blocks[1:]
    #display(current_block)
    
    MEMORY[block_id] = parent_to_calc
        
    i += 1
    
tex, _ = atw_latex_formatter(main_block)
display_latex(tex)

In [ ]:
ArithmeticExpr = ANTLR(r"""
grammar arithmetic;
ignoblebug: expr;

s : expr EOF;



expr                                                      // TODO: parentesizzazione semplice efficiente
   : (PLUS | MINUS)? (INT|RAT|REAL)   # atomExpr
   | <assoc=right>expr POW expr           # powExpr
   | expr FRACT expr                      # FractExpr
   | (PLUS | MINUS) expr                  # unaryExpr
   | expr (TIMES | DIV) expr              # divProdExpr
   | expr (PLUS | MINUS) expr             # addSubExpr
   | ROUND_L_BRACK expr ROUND_R_BRACK     # roundBlockExpr
   | CURLY_L_BRACK expr CURLY_R_BRACK     # curlyBlockExpr
   | SQUARE_L_BRACK expr SQUARE_R_BRACK   # squareBlockExpr
   | ROUND_L_BRACK expr ROUND_R_BRACK     # roundBlockExpr
   | SUBEXP_L_BRACK expr SUBEXP_R_BRACK   # subExpr
   ;


INT
   : NUM 
   ;
   
RAT
    : NUM FRACT NUM
    ;

REAL
   : NUM ('.' NUM +)? (E SIGN? NUM)?
   ;
   
NUM
   :  ('0' .. '9') + (('0' .. '9') +)?
   ;   


fragment E
   : 'E' | 'e'
   ;
   
fragment SIGN
   : ('+' | '-')
   ;
   
CURLY_L_BRACK : '{' ;
CURLY_R_BRACK : '}' ;
SQUARE_L_BRACK : '[' ;
SQUARE_R_BRACK : ']' ;
ROUND_L_BRACK : '(' ;
ROUND_R_BRACK : ')' ;
SUBEXP_L_BRACK : '<' ;
SUBEXP_R_BRACK : '>' ;
PLUS   : '+' ;
MINUS  : '-' ;
TIMES  : 'x' ;
DIV    : ':' ;
POINT  : '.' ;
POW    : '^' ;
FRACT  : '/' ;

WS
   : [ \r\n\t] + -> skip
   ;
   

""")

In [ ]:
expr = """
1 + (2 x ((3 + 4 : 2) + 5) x 6) - (7 + (1 + 8) : 3) x (9 : (2 + 1) + 2)
"""

expr = '3 + 13 x -<1 + 3>/<2^<1+4> - 6>'

ptree = ArithmeticExpr.tree(expr, 's')
ptree

In [ ]:
arith2ast = AnnotatedTreeWalker('name')


PRIORITY = {
    'graphBlockExpr' : 8,
    'squareBlockExpr' : 7,
    'roundBlockExpr' : 6,
    'powExpr' : 5,
    'unaryExpr' : 4,
    'fractExpr' : 3,
    'divProdExpr' : 2,
    'addSubExpr' : 1
}

@arith2ast.register
def RAT(visit, ptree):
    return Fraction(ptree.root['value'])

@arith2ast.register
def INT(visit, ptree):
    return int(ptree.root['value'])

@arith2ast.register
def REAL(visit, ptree):
    return float(ptree.root['value'])

@arith2ast.register
def atomExpr(visit, ptree):
    if len(ptree.children) > 1:
        value = -visit(ptree.children[1])
    else:
        value = visit(ptree.children[0])
    return Tree({'type': 'atomExpr', 'value': value, 'grade': 0}, []), 0

@arith2ast.register
def addSubExpr(visit, ptree):
    left, op, right = ptree.children
    left, left_priority = visit(left)
    right, right_priority = visit(right)
    
    grade = max(left.root['grade'], right.root['grade'])
    priority = max(left_priority, right_priority, PRIORITY['addSubExpr'])

    return Tree({'type': 'addSubExpr', 'op': op.root['value'], 'priority': priority, 'grade': grade}, [left, right]), priority


@arith2ast.register
def divProdExpr(visit, ptree):
    left, op, right = ptree.children
    left, left_priority = visit(left)
    right, right_priority = visit(right)
    
    grade = max(left.root['grade'], right.root['grade'])
    priority = max(left_priority, right_priority, PRIORITY['divProdExpr'])    

    return Tree({'type': 'divProdExpr', 'op': op.root['value'], 'priority': priority, 'grade': grade}, [left, right]), priority


@arith2ast.register
def powExpr(visit, ptree):
    left, _, right = ptree.children
    left, left_priority = visit(left)
    right, right_priority = visit(right)
    
    grade = max(left.root['grade'], right.root['grade'])
    priority = max(left_priority, right_priority, PRIORITY['powExpr'])  
    
    return Tree({'type': 'powExpr', 'priority': priority, 'grade': grade}, [left, right]), priority

@arith2ast.register
def unaryExpr(visit, ptree):
    op, subexpr = ptree.children
    subexpr, subexpr_priority = visit(subexpr)
    
    grade = subexpr.root['grade']
    priority = max(subexpr_priority, PRIORITY['unaryExpr'])  
    
    return Tree({'type': 'unaryExpr', 'op': op.root['value'],'priority': priority, 'grade': grade}, [subexpr]), subexpr_priority



@arith2ast.register
def fractExpr(visit, ptree):
    left, _, right = ptree.children
    left, left_priority = visit(left)
    right, right_priority = visit(right)
    
    grade = max(left.root['grade'], right.root['grade'])
    priority = max(left_priority, right_priority, PRIORITY['fractExpr'])   
    
    return Tree({'type': 'fractExpr', 'priority': priority, 'grade': grade}, [left, right]), priority


@arith2ast.register
def roundBlockExpr(visit, ptree):
    _, subexpr, _ = ptree.children
    subexpr, subexpr_priority = visit(subexpr)
    
    grade = subexpr.root['grade'] + 1
    priority = PRIORITY['roundBlockExpr']   
        
    if subexpr_priority > priority:
        raise BaseException(f'Annidamento scorretto in posizione: {ptree.root["src"]}')
        
    return Tree({'type': 'roundBlockExpr', 'priority': priority, 'grade': grade}, [subexpr]), priority


@arith2ast.register
def squareBlockExpr(visit, ptree):
    _, subexpr, _ = ptree.children
    subexpr, subexpr_priority = visit(subexpr)
    
    grade = subexpr.root['grade'] + 1
    priority = PRIORITY['squareBlockExpr']   
        
    if subexpr_priority > priority:
        raise BaseException(f'Annidamento scorretto in posizione: {ptree.root["src"]}')
        
    return Tree({'type': 'squareBlockExpr', 'priority': priority, 'grade': grade}, [subexpr]), priority

    
@arith2ast.register
def curlyBlockExpr(visit, ptree):
    _, subexpr, _ = ptree.children
    subexpr, subexpr_priority = visit(subexpr)
    
    grade = subexpr.root['grade'] + 1
    priority = PRIORITY['graphBlockExpr']   

    
    return Tree({'type': 'graphBlockExpr', 'priority': priority, 'grade': grade}, [subexpr]), priority
    #depth = depth + 1
    #subexpr.root['depth'] = depth
    #return subexpr, depth
    

@arith2ast.register
def subExpr(visit, ptree):
    _, subexpr, _ = ptree.children
    return visit(subexpr)
    #depth = depth + 1
    #subexpr.root['depth'] = depth
    #return subexpr, depth
    
@arith2ast.register
def s(visit, ptree):
    ast, _ = visit(ptree.children[0])
    return ast


In [ ]:
ast = arith2ast(ptree)
ast

In [ ]:
def src2ast(source):
    ast = arith2ast(ArithmeticExpr.tree(source, 's'))
    return ast

In [ ]:
atw_priority = AnnotatedTreeWalker('type', AnnotatedTreeWalker.RECOURSE_CHILDREN)

ARITH_OP = {
    '+' : add,
    '-' : sub,
    'x' : mul,
    ':' : Fraction,
    '/' : Fraction
}



def find_more_priority_child(ast):
    priorities = [child.root['priority'] if 'priority' in child.root else -1 for child in ast.children]
    grades = [child.root['grade'] for child in ast.children]
    l = list(zip(priorities, grades))
    return l.index(max(l, key=lambda x: float(f'{x[0]}.{x[1]}')))


def is_calculable(ast):
    return ast.root['type'] == 'atomExpr' or all(child.root['type'] == 'atomExpr' for child in ast.children)

@atw_priority.register
def addSubExpr(visit, ast):
    if is_calculable(ast):  
        ast.root['_calc'] = 'next'
    else:
        index_priority_child = find_more_priority_child(ast)
        visit(ast.children[index_priority_child])

        child_types = set(child.root['type'] for child in ast.children)
        if not(child_types ^ {'addSubExpr', 'atomExpr'}) and all(is_calculable(child) for child in ast.children) :
            ast.root['_calc'] = 'next'
            del ast.children[index_priority_child].root['_calc']
        
    
    

@atw_priority.register
def divProdExpr(visit, ast):
    if is_calculable(ast):  
        ast.root['_calc'] = 'next'
    else:
        index_priority_child = find_more_priority_child(ast)
        visit(ast.children[index_priority_child])

        child_types = set(child.root['type'] for child in ast.children)
        if not(child_types ^ {'divProdExpr', 'atomExpr'}) and all(is_calculable(child) for child in ast.children) :
            ast.root['_calc'] = 'next'
            del ast.children[index_priority_child].root['_calc']


@atw_priority.register
def powExpr(visit, ast):
    if is_calculable(ast):  
        ast.root['_calc'] = 'next'
    else:
        index_priority_child = find_more_priority_child(ast)
        visit(ast.children[index_priority_child])


@atw_priority.register
def unaryExpr(visit, ast):
    if is_calculable(ast):  
        ast.root['_calc'] = 'next'
    else:
        index_priority_child = find_more_priority_child(ast)
        visit(ast.children[index_priority_child])


@atw_priority.register
def fractExpr(visit, ast):
    if is_calculable(ast):  
        ast.root['_calc'] = 'next'
    else:
        index_priority_child = find_more_priority_child(ast)
        visit(ast.children[index_priority_child])

@atw_priority.register
def roundBlockExpr(visit, ast):
    visit(ast.children[0])


@atw_priority.register
def squareBlockExpr(visit, ast):
    visit(ast.children[0])


@atw_priority.register
def graphBlockExpr(visit, ast):
    visit(ast.children[0])

In [ ]:
a = {2,3, 5}
b = {1, 2, 3}
a ^ b

In [ ]:
def step(ast):
    res, _ = atw_eval(ast)
    return res

In [ ]:
atw_step = AnnotatedTreeWalker('type')

@atw_step.catchall
def catchall(visit, ast):
    if ast.root['type'] == 'atomExpr': 
        root = ast.root.copy()
        root.pop('_calc', None)
        return Tree(root, [])
    if '_calc' in ast.root and ast.root['_calc'] == 'next': 
        value = atw_eval(ast)
        return Tree({'type': 'atomExpr', 'value': value, 'grade': 0, '_calc': 'last'}, [])
    
    children = [visit(child) for child in ast.children]
    grade = max(child.root['grade'] for child in children)
    priority = max([child.root['priority'] for child in children if 'priority' in child.root] + [PRIORITY[ast.root['type']]])   
    
    root = ast.root.copy()
    root['grade'] = grade
    root['priority'] = priority
    res = Tree(root, children)
    
    if ast.root['type'] in {'roundBlockExpr', 'squareBlockExpr', 'graphBlockExpr'}:
        if is_calculable(res):
            return children[0]
        res.root['grade'] += 1
    
    return res
    
atw_eval = AnnotatedTreeWalker('type')

ARITH_OP = {
    '+' : add,
    '-' : sub,
    'x' : mul,
    ':' : Fraction,
    '/' : Fraction
}

@atw_eval.register
def atomExpr(visit, ast):
    return ast.root['value']

@atw_eval.register
def addSubExpr(visit, ast):
    op = ast.root['op']
    
    left, right = ast.children
    left = visit(left)
    right = visit(right)
        
    value = ARITH_OP[op](left, right)
    return value

@atw_eval.register
def divProdExpr(visit, ast):
    op = ast.root['op']
    
    left, right = ast.children
    left = visit(left)
    right = visit(right)
        
    value = ARITH_OP[op](left, right)
    return value


@atw_eval.register
def powExpr(visit, ast):
    left, right = ast.children
    left = visit(left)
    right = visit(right)

    value = pow(left, right)
    return value


@atw_eval.register
def unaryExpr(visit, ast):
    subexpr = visit(ast.children[0])
        
    value = subexpr
    if ast.root['op'] == '-':
        value = -value
    return value


@atw_eval.register
def fractExpr(visit, ast):
    left, right = ast.children
    left = visit(left)
    right = visit(right)

    value = Fraction(left, right)
    return value

@atw_eval.register
def roundBlockExpr(visit, ast):
    child = visit(ast.children[0])
    return child


@atw_eval.register
def squareBlockExpr(visit, ast):
    child = visit(ast.children[0])
    return child

@atw_eval.register
def graphBlockExpr(visit, ast):
    child = visit(ast.children[0])
    return child


In [ ]:
def display_latex(tex):
    display(Latex(fr"""\begin{{align}}{tex}\end{{align}}"""))

In [ ]:
atw_latex_formatter = AnnotatedTreeWalker('type')


@atw_latex_formatter.register
def atomExpr(visit, ast):
    fr = Fraction(ast.root['value'])
    res = str(fr)
    if fr.denominator != 1:
        res = f'\\frac{{{fr.numerator}}} {{{fr.denominator}}}'
    if '_calc' in ast.root and ast.root['_calc'] == 'last':
        res = f'\\color{{green}}{{\\boxed{{{res}}}}}'
    return res, False
    
@atw_latex_formatter.register
def addSubExpr(visit, ast):
    op = ast.root['op']
    
    left, right = ast.children
    left, left_calculable = visit(left)
    right, right_calculable = visit(right)
    
    if '_calc' in ast.root and ast.root['_calc'] == 'next':
        return f'\\color{{red}}{{\\boxed{{{left}{op}{right}}}}}', True
    return f'{left} {op} {right}', left_calculable or right_calculable
    
    
@atw_latex_formatter.register
def divProdExpr(visit, ast):
    op = ast.root['op']
    
    left, right = ast.children
    
    left, left_calculable = visit(left)
    right, right_calculable = visit(right)

    if op == 'x':
        op = '\\times'
        
    if '_calc' in ast.root and ast.root['_calc'] == 'next':
        return f'\\color{{red}}{{\\boxed{{{left}{op}{right}}}}}', True
    return f'{left} {op} {right}', left_calculable or right_calculable


@atw_latex_formatter.register
def powExpr(visit, ast):
    left, right = ast.children
    
    left, left_calculable = visit(left)
    right, right_calculable = visit(right)
    
    if '_calc' in ast.root and ast.root['_calc'] == 'next':
        return f'\\color{{red}}{{\\boxed{{{left}^{right}}}}}', True
    return f'{left}^{right}', left_calculable or right_calculable

@atw_latex_formatter.register
def unaryExpr(visit, ast):
    child, child_calculable = visit(ast.children[0])
    
    if '_calc' in ast.root and ast.root['_calc'] == 'next':
        return f'\\color{{red}}{{\\boxed{{-{child}}}}}', True
    return f'-{child}', child_calculable

@atw_latex_formatter.register
def fractExpr(visit, ast):
    
    left, right = ast.children
    
    left, left_calculable = visit(left)
    right, right_calculable = visit(right)
    
    if '_calc' in ast.root and ast.root['_calc'] == 'next':
        return f'\\color{{red}}{{\\boxed{{\\frac{{{left}}} {{{right}}}}}}}', True
    return f'\\frac{{{left}}} {{{right}}}', left_calculable or right_calculable


@atw_latex_formatter.register
def roundBlockExpr(visit, ast):
    child, calc = visit(ast.children[0])
    if calc:
        return f'\\color{{blue}}{{\\left({child}\\right)}}', False
    return f'\\left({child}\\right)', False


@atw_latex_formatter.register
def squareBlockExpr(visit, ast):
    child, calc = visit(ast.children[0])
    if calc:
        return f'\\color{{blue}}{{\\left({child}\\right)}}', False
    return f'\\left({child}\\right)', False


@atw_latex_formatter.register
def graphBlockExpr(visit, ast):
    child, calc = visit(ast.children[0])
    if calc:
        return f'\\color{{blue}}{{\\left({child}\\right)}}', False
    return f'\\left({child}\\right)', False



In [ ]:
expr = '1 x 8 x 7 x (8 + 7) + (9 x 7 + 8 - (3 + 2+ 3))) - (8 + 9 + 8 + (4))'
ex1 = '1 + (2 x ((3 + 4 : 2) + 5) x 6) - (7 + (1 + 8) : 3) x(9 : (2 + 1) +2)'
ex2 = '<2+3>/4/<5+6>/7'
ex3 = '2^<3+4^5:2^7>'
ex4 = '3 + 13 x -<1 + 3>/<2^<1+4> - 6>'
ex5a = '3 / < 4 + 2 > + 2 x { 3 : ( 4 + 5 ) + 2 ^ < 6 x 1/2 > } - < 2 x [ 5 x ( 3 + 4 ) + 5 ] - 1 > / 6'
ex5b = '3 / < 4 + 2 > + 2 x ( 3 : ( 4 + 5 ) + 2 ^ < 6 x 1/2 > ) - < 2 x ( 5 x ( 3 + 4 ) + 5 ) - 1 > / 6'
#expr = ex1
expr = '4/5+5x5^-2'

ast = src2ast(expr)
#step(ast)
#step(ast)
count = 0
while ast.children:
    #display(ast)
    atw_priority(ast)
    tex, _ = atw_latex_formatter(ast)
    display_latex(tex)
    ast = atw_step(ast)
    count += 1
    #print('-' * 100)



In [ ]:
expr = ex5a
ast = src2ast(expr)
atw_priority(ast)

text, _ = atw_latex_formatter(ast)
text
display_latex(text)


In [ ]:
display(Latex(fr"""
\begin{{align}}
{text}
\end{{align}}
"""))


In [ ]:
True or False

In [ ]:
atw_eval = AnnotatedTreeWalker('type')

ARITH_OP = {
    '+' : add,
    '-' : sub,
    'x' : mul,
    ':' : Fraction,
    '/' : Fraction
}



def find_more_priority_child(ast):
    priorities = [child.root['priority'] if 'priority' in child.root else -1 for child in ast.children]
    grades = [child.root['grade'] for child in ast.children]
    l = list(zip(priorities, grades))
    return l.index(max(l, key=lambda x: float(f'{x[0]}.{x[1]}')))


def is_calculable(ast):
    return ast.root['type'] == 'atomExpr' or all(child.root['type'] == 'atomExpr' for child in ast.children)


@atw_eval.register
def atomExpr(visit, ast):
    grade = 0
    return Tree({'type': ast.root['type'], 'value': ast.root['value'], 'grade': grade}, []), 0

@atw_eval.register
def addSubExpr(visit, ast):
    op = ast.root['op']
    
    if is_calculable(ast):  
        left, right = ast.children
        left, _ = visit(left)
        right, _ = visit(right)
        
        value = ARITH_OP[op](left.root['value'], right.root['value'])
        grade = 0
        return Tree({'type': 'atomExpr', 'value': value, 'grade': grade}, []), 0
    
    
    index_priority_child = find_more_priority_child(ast)
    children = ast.children.copy()
    children[index_priority_child], _ = visit(ast.children[index_priority_child])
    
    grade = max([child.root['grade'] for child in children if 'grade' in child.root] + [0])
    
    priority = max([child.root['priority'] for child in children if 'priority' in child.root] + [PRIORITY['addSubExpr']])
    
    res = Tree({'type': 'addSubExpr', 'op': op, 'grade' : grade, 'priority': priority}, children)
    
    if is_calculable(res) and ast.children[index_priority_child].root['type'] == 'addSubExpr':
        return visit(res)
    return res, priority
    
    
    

@atw_eval.register
def divProdExpr(visit, ast):
    op = ast.root['op']
    
    if is_calculable(ast):  
        left, right = ast.children
        left, _ = visit(left)
        right, _ = visit(right)

        value = ARITH_OP[op](left.root['value'], right.root['value'])
        grade = 0
        return Tree({'type': 'atomExpr', 'value': value, 'grade': grade}, []), 0
    
    
    index_priority_child = find_more_priority_child(ast)
    children = ast.children.copy()
    children[index_priority_child], _ = visit(ast.children[index_priority_child])
    
    grade = max([child.root['grade'] for child in children if 'grade' in child.root] + [0])
    
    priority = max([child.root['priority'] for child in children if 'priority' in child.root] + [PRIORITY['divProdExpr']])

    res = Tree({'type': 'divProdExpr', 'op': op, 'grade' : grade, 'priority': priority}, children)
    
    if is_calculable(res) and ast.children[index_priority_child].root['type'] == 'divProdExpr':
        return visit(res)
    return res, priority


@atw_eval.register
def powExpr(visit, ast):
    if is_calculable(ast):  
        left, right = ast.children
        left, _ = visit(left)
        right, _ = visit(right)

        value = pow(left.root['value'], right.root['value'])
        grade = 0
        return Tree({'type': 'atomExpr', 'value': value, 'grade': grade}, []), 0
    
    
    index_priority_child = find_more_priority_child(ast)
    children = ast.children.copy()
    children[index_priority_child], _ = visit(ast.children[index_priority_child])
    
    grade = max([child.root['grade'] for child in children if 'grade' in child.root] + [0])
    
    priority = max([child.root['priority'] for child in children if 'priority' in child.root] + [PRIORITY['powExpr']])
    return Tree({'type': 'powExpr', 'grade' : grade, 'priority': priority}, children), priority



@atw_eval.register
def unaryExpr(visit, ast):
    if is_calculable(ast):  
        subexpr, _ = visit(ast.children[0])
        
        value = subexpr.root['value']
        if ast.root['op'] == '-':
            value = -value
        grade = 0
        return Tree({'type': 'atomExpr', 'value': value, 'grade': grade}, []), 0
    
    
    
    subexpr, grade = visit(ast.children[0])
    
    if 'priority' in subexpr.root:
    else:    
        priority = PRIORITY['unaryExpr']
    return Tree({'type': 'unaryExpr', 'op': ast.root['op'],'grade' : grade, 'priority': priority}, [subexpr]), priority


@atw_eval.register
def fractExpr(visit, ast):
    if is_calculable(ast):  
        left, right = ast.children
        left, _ = visit(left)
        right, _ = visit(right)

        value = Fraction(left.root['value'], right.root['value'])
        grade = 0
        return Tree({'type': 'atomExpr', 'value': value, 'grade': grade}, []), 0
    
    
    index_priority_child = find_more_priority_child(ast)
    children = ast.children.copy()
    children[index_priority_child], _ = visit(ast.children[index_priority_child])
    
    grade = max([child.root['grade'] for child in children if 'grade' in child.root] + [0])
    
    priority = max([child.root['priority'] for child in children if 'priority' in child.root] + [PRIORITY['fractExpr']])
    return Tree({'type': 'fractExpr', 'grade' : grade, 'priority': priority}, children), priority

@atw_eval.register
def roundBlockExpr(visit, ast):
    child, child_priority = visit(ast.children[0])
    grade_block = child.root['grade'] + 1
    priority = PRIORITY['roundBlockExpr']
    res = Tree({'type': 'roundBlockExpr', 'grade': grade_block, 'priority': priority}, [child])
    if is_calculable(res):
        return child, child_priority
    return res, priority


@atw_eval.register
def squareBlockExpr(visit, ast):
    child, child_priority = visit(ast.children[0])
    grade_block = child.root['grade'] + 1
    priority = PRIORITY['squareBlockExpr']
    res = Tree({'type': 'squareBlockExpr', 'grade': grade_block, 'priority': priority}, [child])
    if is_calculable(res):
        return child, child_priority
    return res, priority

@atw_eval.register
def graphBlockExpr(visit, ast):
    child, child_priority = visit(ast.children[0])
    grade_block = child.root['grade'] + 1
    priority = PRIORITY['graphBlockExpr']
    res = Tree({'type': 'graphBlockExpr', 'grade': grade_block, 'priority': priority}, [child])
    if is_calculable(res):
        return child, child_priority
    return res, priority
